In [1]:
import sys
# set python syspath to point out location of our self-writing module
sys.path.append("/home/ponshane/work_dir/CLTM/src/codebase/")
from helper import *

import configparser
import pymongo
from pymongo import MongoClient
from gensim.models import KeyedVectors
import pandas as pd
import os

In [2]:
### init and read config
config = configparser.ConfigParser()
config.read('../config.ini')

MongoDB = config["MLDoc"]["Database"]
MongoUser = config["MLDoc"]["User"]
MongoPW = config["MLDoc"]["PW"]

###連接MONGO
uri = "mongodb://" + MongoUser + ":" + MongoPW + "@140.117.69.70:30241/" +\
MongoDB + "?authMechanism=SCRAM-SHA-1"

client = MongoClient(uri)
db = client.MLDoc

# tag train/dev/test on Chinese Corpus

In [30]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/chinese.train.1000", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.Chinese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"train"}})
    
fo.close()

In [31]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/chinese.dev", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.Chinese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"dev"}})
    
fo.close()

In [32]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/chinese.test", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.Chinese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"test"}})
    
fo.close()

# tag train/dev/test on English Corpus

In [37]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/english.train.1000", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.English.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"train"}})
    
fo.close()

In [38]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/english.dev", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.English.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"dev"}})
    
fo.close()

In [39]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/english.test", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.English.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"test"}})
    
fo.close()

# tag train/dev/test on Japanese Corpus

In [40]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/japanese.train.1000", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.Japanese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"train"}})
    
fo.close()

In [41]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/japanese.dev", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.Japanese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"dev"}})
    
fo.close()

In [42]:
fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/japanese.test", "r")
 
for line in fo.readlines():
    
    line = line.split("-")
    # 確認是否為 folder-docID 格式
    assert len(line) == 2

    status = db.Japanese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
                       {"$set": {"Data_type":"test"}})
    
fo.close()

# select data where Data_type = train or test or dev
* only 6000*2 docs in total(1000 train, 1000 dev, 4000 test in each language)
* **English corpus 需要以project方式撈取，否則 word2vec model 有 stop words 此處需要解查** 

In [106]:
#format => [{_id, extracted_text, Class, Data_type, language}, ... ]
corpus = []

# load back model to filter tokens
Concatenated_model = KeyedVectors.load_word2vec_format("../out/MLDoc/Chinese_English_wordvectors.vec")

query = {}
query["Data_type"] = {
    u"$exists": True
}

In [107]:
# For Chinese
cursor = db.Chinese.find(query, no_cursor_timeout=True)
try:
    for doc in cursor:
        each_doc_tokens = []
        for each_sentence in doc["chi_result"]["tokens"]:
            each_doc_tokens += [token for token in each_sentence if token in Concatenated_model.index2word]
        corpus.append({"_id":doc["_id"], "extracted_text":each_doc_tokens, "Class":doc["Class"],
                      "Data_type":doc["Data_type"], "language":"Chinese"})
finally:
    client.close()

In [108]:
# For English
cursor = db.English.find(query, no_cursor_timeout=True)
try:
    for doc in cursor:
        each_doc_tokens = []
        for each_sentence in doc["nested_token_list"]:
            #each_doc_tokens += [token["lemma_"] for token in each_sentence if token["lemma_"] in Concatenated_model.index2word]
            tokens_from_each_sentence = project_function_for_every_document(each_sentence, want_stop=False,
                                                                        want_alpha=True, want_lemma=True,
                                                                        accept_pos = ["NOUN", "VERB"],
                                                                        use_entity=False)
            tokens_from_each_sentence = [token for token in tokens_from_each_sentence
                                         if token in Concatenated_model.index2word]
            each_doc_tokens += tokens_from_each_sentence
        corpus.append({"_id":doc["_id"], "extracted_text":each_doc_tokens, "Class":doc["Class"],
                      "Data_type":doc["Data_type"], "language":"English"})
finally:
    client.close()

In [ ]:
len(corpus)
corpus[11997:11999]

In [110]:
df = pd.DataFrame(corpus)

In [111]:
print(df.shape)
df.head(5)

(11999, 5)


,Class,Data_type,_id,extracted_text,language
0,CCAT,test,5bf419bdd3d28003f2b4057f,"[路透社, 北京, 日电, 经济, 参考报, 周二, 报导, 目前, 上海, 经营, 人民币...",Chinese
1,MCAT,test,5bf419bdd3d28003f2b40580,"[路透社, 纽约, 日电, 美国, 现货, 债市, 週五, 前景, 乐观, 气氛, 中收, ...",Chinese
2,ECAT,test,5bf419bdd3d28003f2b40585,"[路透社, 东京, 日电, 总研, 週五, 一份, 报告, 指出, 日本, 结关, 贸易顺差...",Chinese
3,ECAT,test,5bf419bdd3d28003f2b4058b,"[路透社, 法兰克福, 日电, 德国, 央行, 週四, 表示, 今日, 委员, 会议, 打算...",Chinese
4,MCAT,test,5bf419bdd3d28003f2b4058d,"[纽元, 美元, 週五, 收盘, 下跌, 超过, 美分, 午后, 持稳, 美分, 交易员, ...",Chinese


In [112]:
df.to_pickle("../out/MLDoc/tagged_englishAndchinese_corpus_pd.pkl")

# Japanese-English Corpus

In [3]:
#format => [{_id, extracted_text, Class, Data_type, language}, ... ]
corpus = []

# load back model to filter tokens
Concatenated_model = KeyedVectors.load_word2vec_format("../out/MLDoc/Japanese_English_wordvectors.vec")

query = {}
query["Data_type"] = {
    u"$exists": True
}

word_set = set(Concatenated_model.index2word)

In [4]:
# For Japanese
cursor = db.Japanese.find(query, no_cursor_timeout=True)
try:
    for doc in cursor:
        each_doc_tokens = []
        # no setence layer in japanese corpus ....
        for each_token in doc["jap_result"]["tokens"]:
            if each_token in word_set:
                each_doc_tokens += [each_token]
        corpus.append({"_id":doc["_id"], "extracted_text":each_doc_tokens, "Class":doc["Class"],
                      "Data_type":doc["Data_type"], "language":"Japanese"})
finally:
    client.close()

In [5]:
# For English
cursor = db.English.find(query, no_cursor_timeout=True)
try:
    for doc in cursor:
        each_doc_tokens = []
        for each_sentence in doc["nested_token_list"]:
            #each_doc_tokens += [token["lemma_"] for token in each_sentence if token["lemma_"] in Concatenated_model.index2word]
            tokens_from_each_sentence = project_function_for_every_document(each_sentence, want_stop=False,
                                                                        want_alpha=True, want_lemma=True,
                                                                        accept_pos = ["NOUN", "VERB"],
                                                                        use_entity=False)
            tokens_from_each_sentence = [token for token in tokens_from_each_sentence
                                         if token in Concatenated_model.index2word]
            each_doc_tokens += tokens_from_each_sentence
        corpus.append({"_id":doc["_id"], "extracted_text":each_doc_tokens, "Class":doc["Class"],
                      "Data_type":doc["Data_type"], "language":"English"})
finally:
    client.close()

In [6]:
len(corpus)

12000

In [7]:
df = pd.DataFrame(corpus)

In [10]:
print(df.shape)
df.head(5)

(12000, 5)


,Class,Data_type,_id,extracted_text,language
0,CCAT,test,5bf41d2bd3d28003f2b46572,"[ヘルシンキ, ロイター, フィンランド, 鉱山, 金属, 通年, 業績, 下回る, する,...",Japanese
1,GCAT,test,5bf41d2bd3d28003f2b4657a,"[ソウル, ロイター, 韓国, 国防省, 同国, 東部, 発見, 韓国, 朝鮮民主主義人民共...",Japanese
2,GCAT,test,5bf41d2bd3d28003f2b46595,"[ウェリントン, ロイター, ニュージーランド, 首相, 総選挙, 連立政権, 樹立, 妥協...",Japanese
3,ECAT,dev,5bf41d2bd3d28003f2b46596,"[英国, ロイター, 英国, クラーク, 蔵相, 発表, する, 予算, により, 公正, ...",Japanese
4,CCAT,test,5bf41d2bd3d28003f2b4659f,"[東京, ロイター, 野村証券, よる, 野村総研, アナリスト, 若林秀樹, ホシデン, ...",Japanese


In [9]:
df.to_pickle("../out/MLDoc/tagged_englishAndjapanese_corpus_pd.pkl")